In [1]:
import pandas_datareader.data as pdr
import pandas as pd
import datetime as dt
import math
import yfinance as yfin


yfin.pdr_override()

In [2]:
start = dt.datetime(1900, 1, 29)
end = dt.datetime.now()
sp500 = pdr.get_data_yahoo('^GSPC', start, end)

sp500

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0
...,...,...,...,...,...,...
2023-09-01,4530.600098,4541.250000,4501.350098,4515.770020,4515.770020,3246260000
2023-09-05,4510.060059,4514.290039,4496.009766,4496.830078,4496.830078,3526250000
2023-09-06,4490.350098,4490.350098,4442.379883,4465.479980,4465.479980,3418850000


In [3]:
sp500.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [4]:
sp500.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24037 entries, 1927-12-30 to 2023-09-08
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       24037 non-null  float64
 1   High       24037 non-null  float64
 2   Low        24037 non-null  float64
 3   Close      24037 non-null  float64
 4   Adj Close  24037 non-null  float64
 5   Volume     24037 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 1.3 MB


In [5]:
sp500_calc = sp500[['Adj Close']]
sp500_calc = sp500_calc.rename(columns={'Adj Close': 'value'})  # change column name to value
sp500_calc['pct_change'] = sp500_calc['value'].pct_change()
sp500_calc

,value,pct_change
Date,,
1927-12-30,17.660000,NaN
1928-01-03,17.760000,0.005663
1928-01-04,17.719999,-0.002252
1928-01-05,17.549999,-0.009594
1928-01-06,17.660000,0.006268
...,...,...
2023-09-01,4515.770020,0.001799
2023-09-05,4496.830078,-0.004194
2023-09-06,4465.479980,-0.006972


In [6]:
sp500_calc['cum_return'] = (1 + sp500_calc['pct_change']).cumprod()
sp500_calc

,value,pct_change,cum_return
Date,,,
1927-12-30,17.660000,NaN,NaN
1928-01-03,17.760000,0.005663,1.005663
1928-01-04,17.719999,-0.002252,1.003397
1928-01-05,17.549999,-0.009594,0.993771
1928-01-06,17.660000,0.006268,1.000000
...,...,...,...
2023-09-01,4515.770020,0.001799,255.706119
2023-09-05,4496.830078,-0.004194,254.633642
2023-09-06,4465.479980,-0.006972,252.858438


In [7]:
total_ret = sp500_calc['cum_return'][-1]
start = dt.datetime(1970, 1, 1) + dt.timedelta(seconds=sp500_calc.index[0].timestamp())
end =  dt.datetime(1970, 1, 1) + dt.timedelta(seconds=sp500_calc.index[-1].timestamp())
period_days = (end - start).days
period_years = period_days / 365.25
period_years

95.69062286105407

In [8]:
CAGR = (total_ret)**(1/period_years)-1
CAGR*100

5.950441331665135

# Including Dividends

In [9]:
# actions = pdr.DataReader('^GSPC', 'yahoo-actions', start, end)  # not working
# actions.head()

In [10]:
sp500_dividend = pd.read_csv("data/multpl/s-p-500-dividend-yield.csv")
sp500_dividend.head(5)

,Date,Value
0,"Sep 8, 2023",1.54%
1,"Sep 30, 2023",1.52%
2,"Aug 31, 2023",1.54%
3,"Jul 31, 2023",1.52%
4,"Jun 30, 2023",1.58%


In [11]:
sp500_dividend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1834 entries, 0 to 1833
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    1834 non-null   object
 1   Value   1834 non-null   object
dtypes: object(2)
memory usage: 28.8+ KB


In [12]:
sp500_dividend['Date'] = sp500_dividend['Date'].astype('<M8[ns]')
sp500_dividend.dtypes

Date     datetime64[ns]
Value            object
dtype: object

In [13]:
sp500_dividend['Value'] = sp500_dividend['Value'].str.replace('%', '')
sp500_dividend['Value'] = sp500_dividend['Value'].astype('float')
sp500_dividend['Value'] = sp500_dividend['Value'] / 100
sp500_dividend.dtypes

Date     datetime64[ns]
Value           float64
dtype: object

In [14]:
sp500_dividend.sample(5)

,Date,Value
116,2014-02-28,0.0197
317,1997-05-31,0.0182
864,1951-10-31,0.0635
1613,1889-05-31,0.0424
1459,1902-03-31,0.0393


In [15]:
sp500_dividend = sp500_dividend.set_index('Date').rename(columns={'Value':'dividend_yield'})

In [16]:
sp500_dividend.head(5)

,dividend_yield
Date,
2023-09-08,0.0154
2023-09-30,0.0152
2023-08-31,0.0154
2023-07-31,0.0152
2023-06-30,0.0158
